In [4]:
import torch.nn as nn


## Creating a class of nn.Module to include RNN with LSTM Units
class LSTM_Model(nn.Module):
    
    def __init__(self, vocab_size, embedding_size, hidden_size, batch_size = 1, n_layers = 1):
        
        super(LSTM_Model, self).__init__()
        
        ## Creating an embedding object to create 'embedding_size' dimensional encoding
        self.encoder = nn.Embedding(vocab_size,embedding_size)
        
        self.lstm = nn.LSTM(embedding_size,hidden_size,n_layers)
        
        ## The output size is assigned as hidden_size * vocab_size (Since the output is one word from the entire vocab)
        self.linear_output = nn.Linear(hidden_size,vocab_size)
        
        self.init_weights()
        
        ## More variables
        self.hidden_size = hidden_size
        self.n_layers = n_layers
    
        
    def init_weights(self):
        random_range = 0.1
        
        ## Setting random values for different layers
        self.encoder.weight.data.uniform_(-random_range, random_range)
        
        self.linear_output.bias.data.zero_()
        self.linear_output.weight.data.uniform_(-random_range, random_range)

    # Defining the forward layer    
    def forward(self, input, hidden):
        
        #Passing through the embedding layer
        embedding_input = self.encoder(input)
        
        lstm_out, hidden = self.lstm(embedding_input, hidden)
        
        decoded = self.linear_output(lstm_out.view(lstm_out.size(0)*lstm_out.size(1), lstm_out.size(2)))
        
        return decoded.view(lstm_out.size(0), lstm_out.size(1), decoded.size(1)), hidden

    
    def init_hidden(self, batch_size):
        
        weight = next(self.parameters())
        
        return (weight.new_zeros(self.n_layers, batch_size, self.hidden_size), \
                    weight.new_zeros(self.n_layers, batch_size, self.hidden_size))